In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 15 12:46:21 2021

@author: preshitadave
"""

#importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt

In [8]:
class vaccine_trial:
    '''
    This class contains methods to create the intial data to be used and the absolute difference of means 
    of the 2 groups created in the constructor class. 
    Method random_groupings uses the 2 inital groups of data by pooling them together and shuffling them 
    together and dividing them into 2 groups. 
    Method permutation_test runs the simulation of random_grouping 'NR' number of times and calculates the p-value.
    '''
    def __init__(self,n):        
        '''
        Creates the test and control data based on the sample size given. 
        1 indicates infected with Hepatitis E, 0 indicates not infected.
        Control indicates unvaccinated pigs and treatment indicates vaccinated pigs. 

        '''
        self.n=n #number of samples 
        self.control = np.random.choice([0,1], self.n, p=[0.5,0.5]) # creation of data of unvaccinated pigs
        self.treatment = np.random.choice([0,1], self.n, p=[0.9,0.1]) # creation of data of vaccinated pigs
        self.observed_statistic = abs(self.treatment.mean() - self.control.mean()) 
        # calculation of observed_statistic absolute mean
        
    def random_groupings(self):
        '''
        This method pools in the data of the 2 initial groups created and randomly shuffles them and divides them 
        into 2 groups with the sample size N 
        '''
        self.all = np.concatenate((self.control, self.treatment), axis=0) # merges all the data together
        temp = self.all.copy() # copy to avoid corruption by user
        np.random.shuffle(temp) # now the columns are shuffled
        g1 = temp[:self.n] # group one
        g2 = temp[self.n:] # group two
        return (g1, g2) 
    
    def permutation_test(self):
        '''
        This method calculates the p-value by simulating random_grouping a certain number of times 
        and returns the p-value. 
        '''
        # obtain distribution under the null hypothesis of no difference
        NR = 2500 # number of times random groupings are performed
        random_statistics = np.zeros(NR) # create an array of zeros to store the difference in means between 2 groups
        for i in range(NR):
            g1, g2 = self.random_groupings()
            random_statistics[i] = abs(g1.mean() - g2.mean()) 
            #calculates absolute value of the difference in means of the 2 randomly generated groups
        p_value = np.sum(random_statistics >= self.observed_statistic) / NR
        return p_value
        

def multiple_test(n):
    '''
    Generates the percentage of simulations that reject the null hypothesis for a particular N. 
    '''
    p_val_list_N=[]
    for i in range(250): # carries out the simulation 250 times 
        test = vaccine_trial(n) # create an object of the vaccine_trial class
        val = test.permutation_test() # call the permutation_test method
        p_val_list_N.append(round(val,2)) # append the p-value to the list 
    p_val_arr_N=np.asarray(p_val_list_N) # convert list to array    
    confidence = np.sum(p_val_arr_N <= 0.05)*100/250 # if True, adds them together and calculates the percentage
    # (which is the percentage as we take 100 simulations into consideration)
    return confidence
    
    
#To generate the percentage of simulations that reject the null hypothesis for different sample sizes
N_total = np.arange(1, 101, 1).tolist() # create a list of sample sizes 'N' 1 to 100
confidence_total=[] # create an empty list to store the percentage of the simulations
for i in N_total:
    confidence = multiple_test(i) # calls the function multiple_test for sample size i
    confidence_total.append(confidence) # appends the value to the list created 


In [34]:
#Plotting the percentage of simulations that reject the null hypothesis for different sample sizes
plt.rcParams["figure.figsize"] = [10, 5]
plt.figure(dpi=1200)
# plt.plot(N_total,confidence_total)
round_to_whole = [round(num) for num in confidence_total] #rounding the confidence values
plt.plot(N_total,round_to_whole)
N = round_to_whole.index(90)
plt.axhline(y=90, c='r', linestyle='dashed')
plt.axvline(x=N+1, c='r', linestyle = 'dashed') # we construct the line at N+1 since we start fron N=1 and not N=0
plt.text(0,92,'Passes the 90% benchmark',rotation=360, fontsize=8, color='green')
plt.text(N-1, 15, 'Sample size at 90% benchmark = {}'.format(N), rotation=90, fontsize=8, color='green')
plt.xlabel('Size of sample size N', fontsize = 9, fontweight='bold')
plt.ylabel('% of simulations that pass the statistical test', fontsize = 9, fontweight='bold')
plt.title('Plot of the percentage of simulations that reject the null hypothesis over different sample sizes', fontsize=10,fontweight='bold');